In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import gymnasium as gym

from itertools import count
import random

In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# basic loop of gymnasium - how to run  environment 
env = gym.make("Ant-v5", render_mode="human")
env = gym.wrappers.RecordEpisodeStatistics(env)
observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [4]:
class ReplayMemory:
    """
        Circular buffer for interactions with environment.
    """
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size
        self.buffer = []
        self._next_idx = 0

    def add(self, item):
        if len(self.buffer) > self._next_idx:
            self.buffer[self._next_idx] = item
        else:
            self.buffer.append(item)
        if self._next_idx == self.buffer_size - 1:
            self._next_idx = 0
        else:
            self._next_idx = self._next_idx + 1

    def sample(self, batch_size):
        indices = [random.randint(0, len(self.buffer) - 1) for _ in range(batch_size)]
        states   = [self.buffer[i][0] for i in indices]
        actions  = [self.buffer[i][1] for i in indices]
        rewards  = [self.buffer[i][2] for i in indices]
        n_states = [self.buffer[i][3] for i in indices]
        dones    = [self.buffer[i][4] for i in indices]
        return states, actions, rewards, n_states, dones

    def __len__(self):
        return len(self.buffer)

memory = ReplayMemory(buffer_size=50000) # default length of episode is 50, so it should fit around 50 episodes. 

In [10]:
class QNet(torch.nn.Module):
    """
        It takes observation-action pair and outputs Q-value for it.
    """
    def __init__(self, actions_space_size: int = 8, observation_stapce_size:int = 105, hidden_size: int=256):
        # defaults set for MuJuCo ant
        super(QNet, self).__init__()

        self.action_space_size = actions_space_size
        self.observation_space_size = observation_stapce_size
        self.hidden_size = hidden_size 
        
        self.net = nn.Sequential(
            nn.Linear(self.observation_space_size + self.action_space_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size * 4),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size * 4, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, 1),
        )
        # we don't apply activation as we want Q value as an ouput

    def forward(self, x):
        # x should be observations and actions concatenated into one vector x
        q = self.net(x)
        return q
    
    
class PolicyNet(nn.Module):
    """
        It takes state as an input and outputs the best action
    """
    def __init__(self, action_space_size: int = 8, observation_stapce_size:int = 105, hidden_size: int=256):
        super().__init__()

        self.action_space_size = action_space_size
        self.observation_space_size = observation_stapce_size
        self.hidden_size = hidden_size 
        
        self.net = nn.Sequential(
            nn.Linear(self.observation_space_size, self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.hidden_size * 4),
            nn.Tanh(),
            nn.Linear(self.hidden_size * 4, self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.action_space_size),
            nn.Tanh()
        )
        # apparently Tanh works better as activation function in agent networks

    def forward(self, s):
        a = self.net(s)
        return a

q_origin_model = QNet().to(device)  # Q_phi
q_target_model = QNet().to(device)  # Q_phi'
_ = q_target_model.requires_grad_(False)  # target model doen't need grad

mu_origin_model = PolicyNet().to(device)  # mu_theta
mu_target_model = PolicyNet().to(device)  # mu_theta'
_ = mu_target_model.requires_grad_(False)  # target model doen't need grad

In [11]:
gamma = 0.99
opt_q = torch.optim.AdamW(q_origin_model.parameters(), lr=0.0005)
opt_mu = torch.optim.AdamW(mu_origin_model.parameters(), lr=0.0005)


def optimize(states, actions, rewards, next_states, dones):
    # Convert to tensor
    states = torch.tensor(states, dtype=torch.float).to(device)
    actions = torch.tensor(actions, dtype=torch.float).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float).to(device).unsqueeze(1)
    next_states = torch.tensor(next_states, dtype=torch.float).to(device)
    dones = torch.tensor(dones, dtype=torch.float).to(device).unsqueeze(1)

    criterion = nn.MSELoss(reduction="sum").to(device)
    # Optimize critic loss
    opt_q.zero_grad()
    q_org = q_origin_model(torch.concat((states, actions), dim=1))
    mu_tgt_next = mu_target_model(next_states)
    q_tgt_next = q_target_model(torch.concat((next_states, mu_tgt_next), dim=1))
    q_tgt = rewards + gamma * (1.0 - dones) * q_tgt_next
    loss_q = criterion(q_org, q_tgt)
    loss_q.backward()
    opt_q.step()

    # element 0 of tensors does not require grad and does not have a grad_fn

    # Optimize actor loss
    opt_mu.zero_grad()
    mu_org = mu_origin_model(states)
    for p in q_origin_model.parameters():
        p.requires_grad = False  # disable grad in q_origin_model before computation
    q_tgt_max = q_origin_model(torch.concat((states, mu_org), dim=1))
    (-q_tgt_max).sum().backward()
    opt_mu.step()
    for p in q_origin_model.parameters():
        p.requires_grad = True  # enable grad again

In [12]:
tau = 0.002

def update_target():
    for var, var_target in zip(q_origin_model.parameters(), q_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data
    for var, var_target in zip(mu_origin_model.parameters(), mu_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data

In [13]:
"""
Ornstein-Uhlenbeck noise implemented by OpenAI
Copied from https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py
"""
class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

ou_action_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(1), sigma=np.ones(1) * 0.05)

# pick up action with Ornstein-Uhlenbeck noise
def pick_sample(s):
    with torch.no_grad():
        s = np.array(s)
        s_batch = np.expand_dims(s, axis=0)
        s_batch = torch.tensor(s_batch, dtype=torch.float).to(device)
        action_det = mu_origin_model(s_batch)
        action_det = action_det.squeeze(dim=0)
        noise = ou_action_noise()
        action = action_det.cpu().numpy() + noise
        action = np.clip(action, -1.0, 1.0)
        return action

In [16]:
batch_size = 64

reward_records = []
for i in range(500):
    # Run episode till done
    state, _ = env.reset()
    done = False
    cum_reward = 0
    while not done:
        action = pick_sample(state)
        state_next, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        memory.add([state, action, reward, state_next, float(done)])
        cum_reward += reward

        # Train (optimize parameters)
        if len(memory) >= batch_size:
            states, actions, rewards, n_states, dones = memory.sample(batch_size)
            optimize(states, actions, rewards, n_states, dones)
            update_target()
        s = state_next

    # Output total rewards in episode (max 500)
    print("Run episode{} with rewards {}".format(i, cum_reward), end="\n")
    reward_records.append(cum_reward)

    # stop if reward mean > 475.0
    if np.average(reward_records[-10:]) > 2000:
        break

print("\nDone")

Run episode0 with rewards 525.954129361965
Run episode1 with rewards -446.2996623604365
Run episode2 with rewards -2517.3954883102388
Run episode3 with rewards -2316.6800278092087
Run episode4 with rewards -1948.1135278185318
Run episode5 with rewards -1897.7847039452236
Run episode6 with rewards -1180.3051388115314
Run episode7 with rewards -1799.9048270918413
Run episode8 with rewards -1692.5315850866998
Run episode9 with rewards -1086.784542977137
Run episode10 with rewards -1352.932412805691
Run episode11 with rewards -2288.8189931916827
Run episode12 with rewards -2398.3149892193337
Run episode13 with rewards -2281.5509581263764
Run episode14 with rewards -2033.883330878749
Run episode15 with rewards -2048.630915396296
Run episode16 with rewards -2768.35453971072
Run episode17 with rewards -2357.669173395878
Run episode18 with rewards -2460.0282710922847
Run episode19 with rewards -2726.073374791215


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# Generate recent 50 interval average
average_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 50:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-49:idx+1]
    average_reward.append(np.average(avg_list))
plt.plot(reward_records)
plt.plot(average_reward)

### Down there, there is boilerplate for later

In [ ]:
class Agent:
    """
    This is going to represent
    """

    def __init__(
        self,
        learning_rate: float,
        batch_size: int,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """
        Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """
        self.dqn = DQN().to(device)
        self.target_dqn = DQN().to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())

        self.optimizer = optim.AdamW(
            self.dqn.parameters(), lr=learning_rate, amsgrad=True
        )
        self.memory = ReplayMemory(10000)

        self.lr = learning_rate
        self.batch_size = batch_size
        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.training_error = []

    def get_action(self, state: np.ndarray) -> int:
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < self.epsilon:
            return torch.tensor(
                [env.action_space.sample()], device=device, dtype=torch.float32
            )

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            with torch.no_grad():
                return self.dqn(state)

    def optimize_model(self):
        """Updates the model"""

        if len(self.memory) < self.batch_size:
            return
        transitions = self.memory.sample(self.batch_size)
        # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
        # detailed explanation). This converts batch-array of Transitions
        # to Transition of batch-arrays.
        batch = Transition(*zip(*transitions))

        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state would've been the one after which simulation ended)
        non_final_mask = torch.tensor(
            tuple(map(lambda s: s is not None, batch.next_state)),
            device=device,
            dtype=torch.bool,
        )
        non_final_next_states = torch.cat(
            [s for s in batch.next_state if s is not None]
        )
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to policy_net
        state_action_values = self.dqn(state_batch).gather(1, action_batch)

        # Compute V(s_{t+1}) for all next states.
        # Expected values of actions for non_final_next_states are computed based
        # on the "older" target_net; selecting their best reward with max(1).values
        # This is merged based on the mask, such that we'll have either the expected
        # state value or 0 in case the state was final.
        next_state_values = torch.zeros(self.batch_size, device=device)
        with torch.no_grad():
            next_state_values[non_final_mask] = (
                self.target_dqn(non_final_next_states).max(1).values
            )
        # Compute the expected Q values
        expected_state_action_values = (
            next_state_values * self.discount_factor
        ) + reward_batch

        # Compute Huber loss
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # In-place gradient clipping
        torch.nn.utils.clip_grad_value_(self.dqn.parameters(), 100)
        self.optimizer.step()

        # save for plotting later on
        self.training_error.append(loss.item())

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

    def update_target_model(self):
        self.target_dqn.load_state_dict(self.dqn.state_dict())

In [ ]:
# hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 64
N_EPISODES = 100
START_EPSILON = 1.0
EPSILON_DECAY = START_EPSILON / (N_EPISODES / 2)  # reduce the exploration over time
FINAL_EPSILON = 0.1

UPDATE_TARGET_EVERY = 10

agent = Agent(
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    initial_epsilon=START_EPSILON,
    epsilon_decay=EPSILON_DECAY,
    final_epsilon=FINAL_EPSILON,
)

In [ ]:

for i_episode in tqdm(range(N_EPISODES)):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    # fancy itertools for counting
    for t in count():
        action = agent.get_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.squeeze(0).cpu().numpy())
        reward = torch.tensor([reward], device=device)

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        agent.memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        agent.optimize_model()

        if i_episode % UPDATE_TARGET_EVERY == 0:
            agent.update_target_model()

        if terminated or truncated:
            agent.decay_epsilon()
            break

In [ ]:
rolling_length = N_EPISODES // 100
fig, axs = plt.subplots(ncols=3, figsize=(12, 5))
axs[0].set_title("Episode rewards")
# compute and assign a rolling average of the data to provide a smoother graph
reward_moving_average = (
    np.convolve(
        np.array(env.return_queue).flatten(), np.ones(rolling_length), mode="valid"
    )
    / rolling_length
)
axs[0].plot(range(len(reward_moving_average)), reward_moving_average)
axs[1].set_title("Episode lengths")
length_moving_average = (
    np.convolve(
        np.array(env.length_queue).flatten(), np.ones(rolling_length), mode="same"
    )
    / rolling_length
)
axs[1].plot(range(len(length_moving_average)), length_moving_average)
axs[2].set_title("Training Error")
training_error_moving_average = (
    np.convolve(np.array(agent.training_error), np.ones(rolling_length), mode="same")
    / rolling_length
)
axs[2].plot(range(len(training_error_moving_average)), training_error_moving_average)
plt.tight_layout()
plt.show()